In [1]:
import collections

In [2]:
smalltestlines = '''########
#..O.O.#
##@.O..#
#...O..#
#.#.O..#
#...O..#
#......#
########

<^^>>>vv<v>>v<<'''.splitlines()

In [3]:
bigtestlines = '''##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^'''.splitlines()

In [4]:
with open('day15input.txt') as fp:
    data = fp.read().splitlines()

## Part 1 ##

In [5]:
def ingest(lines):
    boxes = set()
    walls = set()
    movelines = []
    in_grid = True
    for row, line in enumerate(lines):
        stripped = line.strip()
        if '' == stripped:
            in_grid = False
            continue
        if in_grid:
            for col, c in enumerate(stripped):
                if '#' == c:
                    walls.add((row, col))
                elif '.' == c:
                    continue
                elif 'O' == c:
                    boxes.add((row, col))
                elif '@' == c:
                    startpos = (row, col)
                else:
                    raise ValueError('Bad line: ', stripped)
        else:
            movelines.append(stripped)
    moves = ''.join(movelines)
    return startpos, walls, boxes, moves

In [6]:
dirs = {'<': (0, -1), '>': (0, +1), '^': (-1, 0), 'v': (+1, 0)}

In [7]:
def move(pos, direc, walls, boxes):
    drow, dcol = dirs[direc]
    nextpos = (pos[0]+drow, pos[1]+dcol)
    seen = []
    while True:
        if nextpos in walls:
            # can't push this way
            return pos, walls, boxes
        elif nextpos in boxes:
            seen.append(nextpos)
        else:
            # this space is empty
            break
        nextpos = (nextpos[0]+drow, nextpos[1]+dcol)
    if not seen:
        # no boxes were pushed
        return nextpos, walls, boxes
    # at least one box was pushed
    newpos = seen[0]
    boxes.remove(newpos)
    boxes.add(nextpos)
    return newpos, walls, boxes

In [8]:
def score(boxes):
    return sum(100*box[0]+box[1] for box in boxes)

In [9]:
def part1(lines):
    startpos, walls, boxes, moves = ingest(lines)
    pos = startpos
    for direc in moves:
        pos, walls, boxes = move(pos, direc, walls, boxes)
    return score(boxes)

In [10]:
assert(2028 == part1(smalltestlines))

In [11]:
assert(10092 == part1(bigtestlines))

In [12]:
part1(data)

1349898

## Part 2 ##

In [13]:
def ingest2(lines):
    leftboxes = set()
    walls = set()
    movelines = []
    in_grid = True
    for row, line in enumerate(lines):
        stripped = line.strip()
        if '' == stripped:
            in_grid = False
            continue
        if in_grid:
            col = 0
            for c in stripped:
                if '#' == c:
                    walls.add((row, col))
                    walls.add((row, col+1))
                elif '.' == c:
                    pass
                elif 'O' == c:
                    leftboxes.add((row, col))
                elif '@' == c:
                    startpos = (row, col)
                else:
                    raise ValueError('Bad line: ', stripped)
                col += 2
        else:
            movelines.append(stripped)
    moves = ''.join(movelines)
    return startpos, walls, leftboxes, moves

In [14]:
def rightmove(pos, walls, leftboxes):
    nextpos = (pos[0], pos[1]+1)
    seen = []
    while True:
        if nextpos in walls:
            # can't push
            return pos, walls, leftboxes
        elif nextpos in leftboxes:
            seen.append(nextpos)
            nextpos = (nextpos[0], nextpos[1]+2)
        else:
            break
    if not seen:
        # no boxes pushed
        return nextpos, walls, leftboxes
    # at least one box was pushed
    newpos = seen[0]
    for r,c in seen:
        leftboxes.remove((r,c))
        leftboxes.add((r, c+1))
    return newpos, walls, leftboxes

In [15]:
def leftmove(pos, walls, leftboxes):
    nextpos = (pos[0], pos[1] - 1)
    seen = []
    while True:
        if nextpos in walls:
            # can't push
            return pos, walls, leftboxes
        elif (nextpos[0], nextpos[1]-1) in leftboxes:
            seen.append((nextpos[0], nextpos[1]-1))
            nextpos = (nextpos[0], nextpos[1]-2)
        else:
            break
    if not seen:
        # no boxes pushed
        return nextpos, walls, leftboxes
    # at least one box was pushed
    for r,c in seen:
        leftboxes.remove((r,c))
        leftboxes.add((r, c-1))
    newpos = (pos[0], pos[1]-1)
    return newpos, walls, leftboxes

In [16]:
def upmove(pos, walls, leftboxes):
    nextpos = (pos[0]-1, pos[1])
    if nextpos in walls:
        return pos, walls, leftboxes
    isleftboxabove = nextpos in leftboxes
    isrightboxabove = (nextpos[0], nextpos[1]-1) in leftboxes
    if (not isleftboxabove) and (not isrightboxabove):
        # nothing in the way, just move and return
        return nextpos, walls, leftboxes
    # there's a box above
    seen = collections.defaultdict(set)
    rowabove = nextpos[0]
    if isleftboxabove:
        seen[rowabove].add(nextpos)
    else:
        seen[rowabove].add((nextpos[0], nextpos[1]-1))
    curr_row = rowabove
    rowabove -= 1
    while True:
        for leftbox in seen[curr_row]:
            # start with left side of box, looking up
            c = leftbox[1]
            if (rowabove, c) in leftboxes:
                seen[rowabove].add((rowabove, c))
            elif (rowabove, c-1) in leftboxes:
                seen[rowabove].add((rowabove, c-1))
            # move to right side of box, looking up
            c = leftbox[1]+1
            if (rowabove, c) in leftboxes:
                seen[rowabove].add((rowabove, c))
            elif (rowabove, c-1) in leftboxes:
                seen[rowabove].add((rowabove, c-1))
        if rowabove not in seen:
            # nothing found above, we can break
            break
        curr_row = rowabove
        rowabove -= 1
    # boxes above found, but can it move?
    for row in seen:
        for box in seen[row]:
            aboveleftpos = (box[0]-1, box[1])
            aboverightpos = (box[0]-1, box[1]+1)
            if (aboveleftpos in walls) or (aboverightpos in walls):
                # there's a wall above this box, so no movement
                #print('blocker above box at ', box)
                return pos, walls, leftboxes
    # okay, everything seen can move
    for row in (sorted(seen.keys())):
        for box in seen[row]:
            r, c = box
            leftboxes.remove(box)
            leftboxes.add((r-1, c))
    return nextpos, walls, leftboxes

In [17]:
def downmove(pos, walls, leftboxes):
    nextpos = (pos[0]+1, pos[1])
    if nextpos in walls:
        return pos, walls, leftboxes
    isleftboxbelow = nextpos in leftboxes
    isrightboxbelow = (nextpos[0], nextpos[1]-1) in leftboxes
    if (not isleftboxbelow) and (not isrightboxbelow):
        # nothing in the way, just move and return
        return nextpos, walls, leftboxes
    # there's a box below
    seen = collections.defaultdict(set)
    rowbelow = nextpos[0]
    if isleftboxbelow:
        seen[rowbelow].add(nextpos)
    else:
        seen[rowbelow].add((nextpos[0], nextpos[1]-1))
    curr_row = rowbelow
    rowbelow += 1
    while True:
        for leftbox in seen[curr_row]:
            # start with left side of box, looking down
            c = leftbox[1]
            if (rowbelow, c) in leftboxes:
                seen[rowbelow].add((rowbelow, c))
            elif (rowbelow, c-1) in leftboxes:
                seen[rowbelow].add((rowbelow, c-1))
            # move to right side of box, looking down
            c = leftbox[1]+1
            if (rowbelow, c) in leftboxes:
                seen[rowbelow].add((rowbelow, c))
            elif (rowbelow, c-1) in leftboxes:
                seen[rowbelow].add((rowbelow, c-1))
        if rowbelow not in seen:
            # nothing found below, we can break
            break
        curr_row = rowbelow
        rowbelow += 1
    # boxes below found, but can they move?
    for row in seen:
        for box in seen[row]:
            belowleftpos = (box[0]+1, box[1])
            belowrightpos = (box[0]+1, box[1]+1)
            if (belowleftpos in walls) or (belowrightpos in walls):
                # there's a wall blocking the move; can't move
                #print('blocker below box at ', box)
                return pos, walls, leftboxes
    # okay, everything seen can move
    for row in reversed(sorted(seen.keys())):
        for box in seen[row]:
            r, c = box
            leftboxes.remove(box)
            leftboxes.add((r+1, c))
    return nextpos, walls, leftboxes

In [18]:
dirfuncs = {'<': leftmove, '>': rightmove, '^': upmove, 'v': downmove}

In [19]:
test2lines = '''#######
#...#.#
#.....#
#..OO@#
#..O..#
#.....#
#######

<vv<<^^<<^^'''.splitlines()

In [20]:
def part2(lines):
    pos, walls, leftboxes, moves = ingest2(lines)
    for move in moves:
        dirfunc = dirfuncs[move]
        pos, walls, leftboxes = dirfunc(pos, walls, leftboxes)
    return score(leftboxes)

In [21]:
assert(618 == part2(test2lines))

In [22]:
assert(9021 == part2(bigtestlines))

In [23]:
part2(data)

1376686

In [24]:
def printmap(pos, walls, leftboxes):
    maxrows, maxcols = max(wall[0] for wall in walls), max(wall[1] for wall in walls)
    mymap = []
    for row in range(maxrows+1):
        line = []
        for col in range(maxcols+1):
            if (row, col) in walls:
                line.append('#')
            elif (row, col) == pos:
                line.append('@')
            elif (row, col) in leftboxes:
                line.append('[')
            elif (row, col-1) in leftboxes:
                line.append(']')
            else:
                line.append('.')
        print(''.join(line))

Stole a solution from https://topaz.github.io/paste/#XQAAAQBhAwAAAAAAAAAznIlVuo9qg5SL+E+dSUlMqbFSuX55LQwn13BRRp5b6LvWbsHa2XUd+CrT8JzMVS8rlcuskkNKPfrSKh95KoNk6Vh0DQC6gTsapjgOHkaqs6SICpX4JbylKyAUayT0aApUchDitiYM0jW1Aez74DE46+Ew7cBU0oT9X+ztl3P41yV0Pw89TLCImcNdabfOXcNI1I4ghOe5U1d511Mw+v1FZyqGkR9SYWamQcWTPKTJPqGson/QJXUS1lTkAgR0s+dqbNwbUr1Ljhp7T/W8ZGcy5PosnJ2Xp2GURAhh0LYpCD3mQK27McQSbpUb31cUW8sr5/XmpmZ3qXUSXO3BzdHJfki6P6YrKVdzFKObGbZmqvOcEQReMV2GjvAVpCEO1syB5Inrh2eM+LAgGRgHXHSHyEsehpFtaRKujSoXjjL4LNImtcYgovy7TGxXqBccOl8qxBHUb7jzz4l8GYuTN1jgkWd6uJ7ZgVDY2WsSvIHY3S3izp8jO5CmT+OEtUdrWRkroMZl7XRv/CeOPYAU2QCSE4XGxunK8kC1k99D45GPzmj86DMYi6pNqOm4My/++uT15A== to figure out where my solution had gone wrong

Note that this version uses the ever-popular complex grid instead of (row, col) pairs

In [25]:
grid, moves = open('day15input.txt').read().split('\n\n')

def move(p, d):
    p += d
    if all([
        grid[p] != '[' or move(p+1, d) and move(p, d),
        grid[p] != ']' or move(p-1, d) and move(p, d),
        grid[p] != 'O' or move(p, d), grid[p] != '#']):
            grid[p], grid[p-d] = grid[p-d], grid[p]
            return True


for grid in grid, grid.translate(str.maketrans(
        {'#':'##', '.':'..', 'O':'[]', '@':'@.'})):

    goodpositions = []

    grid = {i+j*1j:c for j,r in enumerate(grid.split())
                     for i,c in enumerate(r)}

    pos, = (p for p in grid if grid[p] == '@')
    print('pos: (', int(pos.imag), ', ', int(pos.real), ')')

    for m in moves.replace('\n', ''):
        dir = {'<':-1, '>':+1, '^':-1j, 'v':+1j}[m]
        copy = grid.copy()

        if move(pos, dir): pos += dir
        else: grid = copy

        goodpositions.append((int(pos.imag), int(pos.real)))

    ans = sum(pos for pos in grid if grid[pos] in 'O[')
    print(int(ans.real + ans.imag*100))

pos: ( 24 ,  24 )
1349898
pos: ( 24 ,  48 )
1376686
